In [1]:
## this notebook will deteck anomalies from the pcap outputs.

import pandas as pd
import numpy as np

from torch import nn

import scipy
import matplotlib.pyplot as plt


from data_extractors import advanced_process, basic_process

import os
import sys

##install pytorch

##install scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.feature_selection import SelectKBest
## train test split import with cross fold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


In [ ]:
current_wd = os.getcwd()

three_days_advanced_raw = advanced_process(f"{current_wd}/pcap-files","three_days_adv.csv") ## this will likely cause errors now because the package is still getting collected in the background.

three_days_basic_raw = basic_process(f"{current_wd}/pcap-files","three_days_basic.csv") 


adv_df = pd.read_csv(three_days_advanced_raw)
basic_df = pd.read_csv(three_days_basic_raw)

Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day3.pcap...
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day2.pcap...
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day1.pcap...
Successfully extracted features from 184 time windows
Data saved to three_days_adv.csv
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day3.pcap...
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day2.pcap...
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day1.pcap...
Successfully extracted features from 184 time windows
Data saved to three_days_basic.csv


In [3]:
get_stats(adv_df)

display(100*"-")

get_stats(basic_df)

'--------------------'

"dataframe's shape is :"

(184, 37)

'--------------------'

"dataframe's columns are :"

...

"dataframe's column stats are :"

'window_start skewness is : -0.4733024976948006'

'window_start kurtosis is : -1.4923749029741296'

An error occured:  'str' object has no attribute 'name'


[]

In [8]:
import pandas as pd
import scipy.stats

def get_stats(dataframe=pd.DataFrame()):
    """
    This function returns the statistics of the given dataframe.

    Parameters:
    dataframe: pd.DataFrame
        The dataframe to be analyzed.
    """
    try:
        display("--------------------")
        display("dataframe's shape is :")
        display(dataframe.shape)
        display("--------------------")
        display("dataframe's columns are :")
        display(dataframe.columns)
        display("--------------------")
        display("dataframe's info is :")
        display(dataframe.info())
        display("--------------------")
        display("dataframe's head is :")
        display(dataframe.head())
        display("--------------------")
        display("dataframe's tail is :")
        display(dataframe.tail())
        display("--------------------")
        display("dataframe's describe is :")
        display(dataframe.describe())
        display("--------------------")
        display("dataframe's null values are :")
        display(dataframe.isnull().sum())
        display("--------------------")
        display("dataframe's unique values are :")
        display(dataframe.nunique())
        display("--------------------")
        display("dataframe's data types are :")
        display(dataframe.dtypes)
        display("--------------------")
        display("dataframe's memory usage is :")
        display(dataframe.memory_usage())
        display("--------------------")

        ## SciPy stats of the dataframe
        display("dataframe's column stats are :")
        
        stats_list = []
        
        for col in dataframe.select_dtypes(include=['number']).columns:  
            stats_list.append({
                "column name": col,
                "skewness": scipy.stats.skew(dataframe[col].dropna()),  # Drop NaN values
                "kurtosis": scipy.stats.kurtosis(dataframe[col].dropna())
            })
        
        framey = pd.DataFrame(stats_list)
        display(framey)
        display("--------------------")

        ## Correlation of the columns
        display("dataframe's numerical correlation is :")
        display(dataframe.corr(numeric_only=True))
        display("--------------------")

        ## Covariance of the columns
        display("dataframe's covariance is :")
        display(dataframe.cov(numeric_only=True))
        display("--------------------")

    except Exception as e:
        print("An error occurred: ", e)
        return []


In [9]:
get_stats(adv_df)

'--------------------'

"dataframe's shape is :"

(184, 37)

'--------------------'

"dataframe's columns are :"

...

,window_id,window_start,packet_count,bytes_per_second,avg_packet_size,packet_size_std,unique_protocols,unique_src_ips,unique_dst_ips,unique_src_ports,...,connection_termination_ratio,retransmission_count,retransmission_rate,internal_external_ratio,ttl_mean,ttl_std,http_get_count,http_post_count,http_other_count,is_baseline
window_id,1.000000,1.000000,-0.145632,-0.148213,0.122459,0.221139,NaN,0.024484,0.009166,0.065912,...,-0.248896,-0.006029,0.280823,0.840247,-0.017844,0.230220,-0.076713,NaN,-0.101127,NaN
window_start,1.000000,1.000000,-0.145632,-0.148213,0.122459,0.221139,NaN,0.024484,0.009166,0.065912,...,-0.248896,-0.006029,0.280823,0.840247,-0.017844,0.230220,-0.076713,NaN,-0.101127,NaN
packet_count,-0.145632,-0.145632,1.000000,0.999717,0.322948,0.066292,NaN,0.323723,0.322352,0.453345,...,0.271912,0.888598,-0.075244,-0.124192,0.731184,0.295434,0.000034,NaN,0.014007,NaN
bytes_per_second,-0.148213,-0.148213,0.999717,1.000000,0.318568,0.061174,NaN,0.309162,0.308078,0.437335,...,0.273850,0.884976,-0.075662,-0.124330,0.729701,0.292994,0.003217,NaN,0.005571,NaN
avg_packet_size,0.122459,0.122459,0.322948,0.318568,1.000000,0.820252,NaN,0.512695,0.463885,0.372481,...,-0.001313,0.357904,-0.141688,0.034163,0.154304,-0.124093,0.180662,NaN,0.055959,NaN
packet_size_std,0.221139,0.221139,0.066292,0.061174,0.820252,1.000000,NaN,0.377086,0.348142,0.247851,...,-0.123637,0.125149,-0.089031,0.104855,-0.063381,-0.160792,0.094086,NaN,0.050078,NaN
unique_protocols,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unique_src_ips,0.024484,0.024484,0.323723,0.309162,0.512695,0.377086,NaN,1.000000,0.976609,0.862643,...,0.086693,0.338167,-0.173472,-0.145531,0.242118,0.162545,0.030227,NaN,0.084592,NaN
unique_dst_ips,0.009166,0.009166,0.322352,0.308078,0.463885,0.348142,NaN,0.976609,1.000000,0.866015,...,0.084842,0.343159,-0.125792,-0.180445,0.236889,0.171784,0.017219,NaN,0.075698,NaN
unique_src_ports,0.065912,0.065912,0.453345,0.437335,0.372481,0.247851,NaN,0.862643,0.866015,1.000000,...,0.003692,0.477085,-0.034545,-0.103713,0.350181,0.271233,0.015360,NaN,0.112938,NaN


'--------------------'

"dataframe's covariance is :"

,window_id,window_start,packet_count,bytes_per_second,avg_packet_size,packet_size_std,unique_protocols,unique_src_ips,unique_dst_ips,unique_src_ports,...,connection_termination_ratio,retransmission_count,retransmission_rate,internal_external_ratio,ttl_mean,ttl_std,http_get_count,http_post_count,http_other_count,is_baseline
window_id,6.292766e+06,3.775660e+08,-9.719394e+06,-1.743919e+08,7.117832e+04,7.993895e+04,0.0,5.232719e+02,2.093597e+02,5.876514e+03,...,-1210.781934,-1.410702e+03,14.742182,240.005580,-8.947273e+02,9.201638e+03,-28.139819,0.0,-37.403243,0.0
window_start,3.775660e+08,2.265396e+10,-5.831636e+08,-1.046352e+10,4.270699e+06,4.796337e+06,0.0,3.139632e+04,1.256158e+04,3.525909e+05,...,-72646.916063,-8.464213e+04,884.530929,14400.334794,-5.368364e+04,5.520983e+05,-1688.389166,0.0,-2244.194583,0.0
packet_count,-9.719394e+06,-5.831636e+08,7.078164e+08,1.247550e+10,1.990804e+06,2.541517e+05,0.0,7.337604e+04,7.808979e+04,4.286681e+05,...,14028.634037,2.205221e+06,-41.892914,-376.224431,3.888430e+05,1.252343e+05,0.132692,0.0,54.943395,0.0
bytes_per_second,-1.743919e+08,-1.046352e+10,1.247550e+10,2.200092e+11,3.462251e+07,4.134845e+06,0.0,1.235455e+06,1.315784e+06,7.290666e+06,...,249091.975578,3.872028e+07,-742.684548,-6640.358979,6.841523e+06,2.189680e+06,220.647040,0.0,385.251935,0.0
avg_packet_size,7.117832e+04,4.270699e+06,1.990804e+06,3.462251e+07,5.368724e+04,2.738773e+04,0.0,1.012081e+03,9.787010e+02,3.067409e+03,...,-0.590090,7.735508e+03,-0.687033,0.901327,7.146624e+02,-4.581249e+02,6.121172,0.0,1.911740,0.0
packet_size_std,7.993895e+04,4.796337e+06,2.541517e+05,4.134845e+06,2.738773e+04,2.076568e+04,0.0,4.629497e+02,4.568081e+02,1.269396e+03,...,-34.550079,1.682231e+03,-0.268486,1.720511,-1.825660e+02,-3.691821e+02,1.982579,0.0,1.063991,0.0
unique_protocols,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0
unique_src_ips,5.232719e+02,3.139632e+04,7.337604e+04,1.235455e+06,1.012081e+03,4.629497e+02,0.0,7.258396e+01,7.576090e+01,2.612063e+02,...,1.432290,2.687438e+02,-0.030929,-0.141179,4.123208e+01,2.206458e+01,0.037657,0.0,0.106260,0.0
unique_dst_ips,2.093597e+02,1.256158e+04,7.808979e+04,1.315784e+06,9.787010e+02,4.568081e+02,0.0,7.576090e+01,8.291016e+01,2.802603e+02,...,1.498111,2.914645e+02,-0.023970,-0.187086,4.311575e+01,2.492240e+01,0.022927,0.0,0.101627,0.0
unique_src_ports,5.876514e+03,3.525909e+05,4.286681e+05,7.290666e+06,3.067409e+03,1.269396e+03,0.0,2.612063e+02,2.802603e+02,1.263179e+03,...,0.254434,1.581665e+03,-0.025694,-0.419718,2.487783e+02,1.535951e+02,0.079829,0.0,0.591827,0.0


'--------------------'